# 1. Thư viện

In [1]:
import os
import json
import re

from langchain_community.embeddings import GPT4AllEmbeddings
from dotenv import load_dotenv
from langchain_qdrant import Qdrant, RetrievalMode
from qdrant_client.models import Filter, FieldCondition, MatchValue

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


# 2. Mô hình embedding

## 2.1. Mô hình Dense embedding

In [2]:
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

model_Dense_Embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
    )

# 3. Kết nối đến Collection

In [3]:
URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")

In [4]:
exist_Qdrant = Qdrant.from_existing_collection(
    embedding = model_Dense_Embedding,
    url = URL_QDRANT,
    api_key = API_QDRANT,
    prefer_grpc=True,
    collection_name = "demo_Luat_Keywords",
	metadata_payload_key="metadata"
)

# 4. Similarity Search kèm Metadata filtering

## 4.1. Hàm trích xuất key words

In [5]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)  # Loại bỏ các dấu câu không mong muốn
    words = text.split()                   # Tách các từ theo khoảng trắng
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

In [6]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

## 4.2. Truy xuất với 1 query

In [7]:
# Query của người dùng
user_Query = "Trong lĩnh vực Dân sự do Quốc hội ban hành thuộc số 10/2022/QH15, \"Thực hiện dân chủ ở cơ sở\" có nghĩa là gì?"

keywords = process_keywords(extract_keywords(user_Query))

# Sử dụng should để lọc dựa trên các từ khóa
user_Filter = Filter(
    should=[
        FieldCondition(
            key="metadata.key_words",  # Trường metadata chứa các từ khóa
            match=MatchValue(value=keyword)
        ) for keyword in keywords
    ]
)

print(user_Filter)

should=[FieldCondition(key='metadata.key_words', match=MatchValue(value='trong'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_words', match=MatchValue(value='lĩnh'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_words', match=MatchValue(value='vực'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_words', match=MatchValue(value='dân'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_words', match=MatchValue(value='sự'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_words', match=MatchValue(value='do'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key=

In [8]:
results = exist_Qdrant.similarity_search_with_score(
    query = user_Query,
    filter = user_Filter,
    k = 5
)

In [9]:
for doc,score in results:
    print("Page_Content:\n",doc.page_content)
    # print("Context:\n",doc.metadata['context'],"\n")
    print("Số hiệu:\n",doc.metadata['so_hieu'])
    print("Score:",score,"\n","-----------------------")

Page_Content:
 Điều 43. Trách nhiệm của Bộ Quốc phòng
1. Ban hành, trình cấp có thẩm quyền ban hành và tổ chức thực hiện văn bản quy phạm pháp luật về phòng thủ dân sự thuộc lĩnh vực quản lý;
Số hiệu:
 18/2023/QH15
Score: 0.842420220375061 
 -----------------------
Page_Content:
 Điều 9. Các hành vi bị nghiêm cấm trong thực hiện dân chủ ở cơ sở
1. Gây khó khăn, phiền hà hoặc cản trở, đe dọa công dân thực hiện dân chủ ở cơ sở.
Số hiệu:
 10/2022/QH15
Score: 0.8407150506973267 
 -----------------------
Page_Content:
 Điều 44. Trách nhiệm của Bộ Công an
1. Ban hành, trình cấp có thẩm quyền ban hành và tổ chức thực hiện văn bản quy phạm pháp luật về phòng thủ dân sự thuộc lĩnh vực quản lý.
Số hiệu:
 18/2023/QH15
Score: 0.8390295505523682 
 -----------------------
Page_Content:
 Điều 50. Trách nhiệm của Bộ Công Thương
1. Ban hành, trình cấp có thẩm quyền ban hành và tổ chức thực hiện văn bản quy phạm pháp luật về phòng thủ dân sự thuộc phạm vi quản lý.
Số hiệu:
 18/2023/QH15
Score: 0.8340569

### 4.2.1. Thử với filer dạng dictionary

In [10]:
# Query của người dùng
user_Query = "Trong lĩnh vực Dân sự do Quốc hội ban hành thuộc số 10/2022/QH15, \"Thực hiện dân chủ ở cơ sở\" có nghĩa là gì?"

keywords = process_keywords(extract_keywords(user_Query))

# Tạo đối tượng dict từ danh sách key_words
filter_data = {
    "key_words": keywords
}

In [11]:
# filter_data = {
#     "so_hieu": "10/2022/QH15"
# }

In [12]:
filter_data

{'key_words': ['trong',
  'lĩnh',
  'vực',
  'dân',
  'sự',
  'do',
  'quốc',
  'hội',
  'ban',
  'hành',
  'thuộc',
  'số',
  '10/2022/QH15',
  'thực',
  'hiện',
  'dân',
  'chủ',
  'ở',
  'cơ',
  'sở',
  'có',
  'nghĩa',
  'là',
  'gì']}

In [13]:
results_test = exist_Qdrant.similarity_search_with_score(
    query = user_Query,
    filter = filter_data,
    k = 5
)

In [14]:
for doc,score in results_test:
    print("Page_Content:\n",doc.page_content)
    # print("Context:\n",doc.metadata['context'],"\n")
    print("Số hiệu:\n",doc.metadata['so_hieu'])
    print("Score:",score,"\n","-----------------------")

# 5. Sinh 3 similarity queries với Gemini

In [15]:
from apikeys_GEMINI import APIKeyManager

from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [16]:
load_dotenv()

MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
# MODEL_GEMINI = "models/gemini-1.5-pro"
if MODEL_GEMINI is None:
    raise ValueError("Environment variable MODEL_GEMINI is not set")
elif not MODEL_GEMINI.startswith("models/"):
    MODEL_GEMINI = f"models/{MODEL_GEMINI}"

APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)

TOP_K = 5
MAX_DOCS_FOR_CONTENT = 5

## 5.1. Hàm RFF (Re-rank)

In [17]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (reciprocal rank fusion)"""
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTENT]]

## 5.2. Hàm sinh câu hỏi

In [18]:
def query_generator(original_query: str, key_manager) -> list[str]:
    """Generate queries from original query"""
    # Câu truy vấn gốc
    query = original_query
    
    # Cập nhật prompt để yêu cầu rõ ràng chỉ trả về 3 câu truy vấn và câu gốc
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant that generates multiple search queries based on a single query."),
            ("human", "Generate 3 search queries related to: {original_query}. Provide each query on a new line, and ensure that only the queries are returned, with no additional explanations or text."),
        ]
    )
    
    model = ChatGoogleGenerativeAI(
        # key api google gemini, nếu test mà bị báo lỗi api core thì lấy api khác trong .env để test
        google_api_key=key_manager.get_next_key(),
        model=MODEL_GEMINI,
        temperature=0
    )
    
    query_generator_chain = (
        prompt | model | StrOutputParser()
    )
    
    # Kết quả sẽ là một chuỗi các câu truy vấn cách nhau bằng dấu xuống dòng
    result = query_generator_chain.invoke({"original_query": query})
    
    # Tách kết quả thành danh sách các câu truy vấn
    generated_queries = result.strip().split('\n')
    
    # Đảm bảo chỉ lấy 3 câu nếu có nhiều hơn 3 câu truy vấn sinh ra
    if len(generated_queries) > 3:
        generated_queries = generated_queries[:3]
    
    # Kết hợp câu gốc với các câu truy vấn sinh ra
    queries = [query] + generated_queries
    
    return queries

## 5.3. Hàm lấy collection

In [19]:
def get_existing_qdrant_collection(collection_name: str, embedding_model, 
                                   qdrant_url: str, qdrant_api_key: str):
    qdrant_instance = Qdrant.from_existing_collection(
        embedding=embedding_model,
        url=qdrant_url,
        api_key=qdrant_api_key,
        prefer_grpc=True,
        collection_name=collection_name,
        metadata_payload_key="metadata"
    )
    return qdrant_instance

## 5.4. Hàm truy vấn

In [20]:
def search_with_qdrant(qdrant_instance, user_query: str, top_k: int = 5):
    keywords = process_keywords(extract_keywords(user_Query))
    
    # Tạo filter dựa trên từ khóa
    user_filter = Filter(
        should=[
            FieldCondition(
                key="metadata.key_words",  # Trường metadata chứa từ khóa
                match=MatchValue(value=keyword)
            ) for keyword in keywords
        ]
    )
    
    # Thực hiện truy vấn similarity search
    results = qdrant_instance.similarity_search_with_score(
        query=user_query,
        filter=user_filter,  # Dùng filter để lọc kết quả
        k=top_k
    )
    
    return results

# 6. Thực hiện truy vấn với similarity queries

In [21]:
def search_and_rerank(user_query: str, key_manager, embedding_model, qdrant_url: str, qdrant_api_key: str):
    """Quy trình tổng hợp: sinh câu hỏi, tìm kiếm và xếp hạng lại"""
    # 1. Sinh ra 4 câu hỏi (gồm câu gốc và 3 câu hỏi tương tự)
    queries = query_generator(user_query, key_manager)

    for q in queries:
        print(q,"\n")
    
    # 2. Lấy collection từ Qdrant
    qdrant_instance = get_existing_qdrant_collection("demo_Luat_Keywords", embedding_model, qdrant_url, qdrant_api_key)
    
    # 3. Thực hiện tìm kiếm cho mỗi câu hỏi và lưu lại kết quả
    all_results = []
    for query in queries:
        results = search_with_qdrant(qdrant_instance, query, top_k=5)  # Mỗi truy vấn lấy top 5 kết quả
        all_results.append(results)
    
    # 4. Áp dụng hàm Re-rank (RRF) để tái xếp hạng 20 documents
    reranked_documents = reciprocal_rank_fusion(all_results)
    
    return reranked_documents  # Trả về top 5 documents sau khi xếp hạng lại

In [22]:
user_query = "Trong lĩnh vực Dân sự do Quốc hội ban hành thuộc số 10/2022/QH15, \"Thực hiện dân chủ ở cơ sở\" có nghĩa là gì?"
embedding_model = model_Dense_Embedding  # Ví dụ về mô hình embedding
qdrant_url = URL_QDRANT
qdrant_api_key = API_QDRANT

top_documents = search_and_rerank(user_query, key_manager, embedding_model, qdrant_url, qdrant_api_key)

for doc in top_documents:
    print(doc)

Trong lĩnh vực Dân sự do Quốc hội ban hành thuộc số 10/2022/QH15, "Thực hiện dân chủ ở cơ sở" có nghĩa là gì? 

Luật số 10/2022/QH15 thực hiện dân chủ ở cơ sở 

Nghị quyết 10/2022/QH15 và dân chủ ở cơ sở 

Giải thích "Thực hiện dân chủ ở cơ sở" theo Luật Dân sự 10/2022/QH15 

[Document(metadata={'Content': '1. Ban hành, trình cấp có thẩm quyền ban hành và tổ chức thực hiện văn bản quy phạm pháp luật về phòng thủ dân sự thuộc lĩnh vực quản lý;', 'Section': None, 'Chapter': 'Chương VI: TRÁCH NHIỆM CỦA CƠ QUAN, TỔ CHỨC VỀ PHÒNG THỦ DÂN SỰ', 'Article-Section': 'Khoản 1.', 'loai_van_ban': 'Luật', 'Mini-Section': None, 'noi_ban_hanh': 'Quốc hội', 'linhvuc_nganh': 'Bộ máy hành chính', 'stt': '9', 'so_hieu': '18/2023/QH15', 'ngay_ban_hanh': '20/06/2023', 'key_words': ['luật', 'quốc', 'hội', '18/2023/QH15', 'bộ', 'máy', 'hành', 'chính', '20/06/2023', '01/07/2024', 'phòng', 'thủ', 'dân', 'sự', 'điều', '43', 'trách', 'nhiệm', 'của', '1', 'ban', 'trình', 'cấp', 'có', 'thẩm', 'quyền', 'và', 'tổ', '

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15100\3142861224.py:12: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


In [23]:
for doc,score in top_documents:
    print("Page_Content:\n",doc.page_content)
    # print("Context:\n",doc.metadata['context'],"\n")
    # print("Số hiệu:\n",doc.metadata['so_hieu'])
    print("Score:",score,"\n")

Page_Content:
 Điều 43. Trách nhiệm của Bộ Quốc phòng
1. Ban hành, trình cấp có thẩm quyền ban hành và tổ chức thực hiện văn bản quy phạm pháp luật về phòng thủ dân sự thuộc lĩnh vực quản lý;
Score: 0.842420220375061 

Page_Content:
 Điều 45. Hiệu lực thi hành
3. Luật Căn cước công dân số 59/2014/QH13 đã được sửa đổi, bổ sung một số điều theo Luật số 68/2020/QH14 hết hiệu lực kể từ ngày Luật này có hiệu lực thi hành.
Score: 0.8288538455963135 

Page_Content:
 Điều 245. Sửa đổi, bổ sung một số điều của Luật Tổ chức chính quyền địa phương số 77/2015/QH13 đã được sửa đổi, bổ sung một số điều theo Luật số 21/2017/QH14, Luật số 47/2019/QH14 và Nghị quyết số 96/2023/QH15

Score: 0.8058667182922363 

Page_Content:
 Điều 2. Giải thích từ ngữ
18. Chủ sở hữu nhà ở là tổ chức, cá nhân có quyền sở hữu nhà ở theo quy định của Luật này.
Score: 0.8396553993225098 

Page_Content:
 Điều 9. Các hành vi bị nghiêm cấm trong thực hiện dân chủ ở cơ sở
1. Gây khó khăn, phiền hà hoặc cản trở, đe dọa công dân 